In [1]:
import argparse
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np


# def main():
fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, default=0, help='status')
parser.add_argument('--model', type=str, default='Ensemble',
                    help='model name, options: [FEDformer, Autoformer, Informer, Ensemble]')

# supplementary config for FEDformer model
parser.add_argument('--version', type=str, default='Fourier',
                    help='for FEDformer, there are two versions to choose, options: [Fourier, Wavelets]')
parser.add_argument('--mode_select', type=str, default='random',
                    help='for FEDformer, there are two mode selection method, options: [random, low]')
parser.add_argument('--modes', type=int, default=64, help='modes to be selected random 64')
parser.add_argument('--L', type=int, default=3, help='ignore level')
parser.add_argument('--base', type=str, default='legendre', help='mwt base')
parser.add_argument('--cross_activation', type=str, default='tanh',
                    help='mwt cross atention activation function tanh or softmax')

# data loader
parser.add_argument('--data', type=str, default='custom', help='dataset type')
parser.add_argument('--root_path', type=str, default='./dataset/btc/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='BTC-Hourly.csv', help='data file')
parser.add_argument('--features', type=str, default='S',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, '
                            'S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='close', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, '
                            'b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--detail_freq', type=str, default='h', help='like freq, but use in predict')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')
# parser.add_argument('--cross_activation', type=str, default='tanh'

# model define
parser.add_argument('--enc_in', type=int, default=1, help='encoder input size')
parser.add_argument('--dec_in', type=int, default=1, help='decoder input size')
parser.add_argument('--c_out', type=int, default=1, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', default=[24], help='window size of moving average')
parser.add_argument('--factor', type=int, default=3, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=0, help='data loader num workers')
parser.add_argument('--itr', type=int, default=1, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=100, help='train epochs')
parser.add_argument('--batch_size', type=int, default=1, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1', help='device ids of multi gpus')

args = parser.parse_args([])
args.task_id = args.data
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.dvices = args.devices.replace(' ', '')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

print('Args in experiment:')
print(args)

# Exp = Exp_Main

# if args.is_training:
#     for ii in range(args.itr):
        # setting record of experiments
# setting = '{}_{}_{}_modes{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
#     args.task_id,
#     args.model,
#     args.mode_select,
#     args.modes,
#     args.data,
#     args.features,
#     args.seq_len,
#     args.label_len,
#     args.pred_len,
#     args.d_model,
#     args.n_heads,
#     args.e_layers,
#     args.d_layers,
#     args.d_ff,
#     args.factor,
#     args.embed,
#     args.distil,
#     args.des,
#     0)

            # exp = Exp(args)  # set experiments
            # print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
            # exp.train(setting)

            # print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            # exp.test(setting)

            # if args.do_predict:
            #     print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            #     exp.predict(setting, True)

            # torch.cuda.empty_cache()
    # else:
    #     ii = 0
    #     setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(args.task_id,
    #                                                                                                   args.model,
    #                                                                                                   args.data,
    #                                                                                                   args.features,
    #                                                                                                   args.seq_len,
    #                                                                                                   args.label_len,
    #                                                                                                   args.pred_len,
    #                                                                                                   args.d_model,
    #                                                                                                   args.n_heads,
    #                                                                                                   args.e_layers,
    #                                                                                                   args.d_layers,
    #                                                                                                   args.d_ff,
    #                                                                                                   args.factor,
    #                                                                                                   args.embed,
    #                                                                                                   args.distil,
    #                                                                                                   args.des, ii)

    #     exp = Exp(args)  # set experiments
    #     print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    #     exp.test(setting, test=1)
    #     torch.cuda.empty_cache()


# if __name__ == "__main__":
#     main()


Args in experiment:
Namespace(is_training=0, model='Ensemble', version='Fourier', mode_select='random', modes=64, L=3, base='legendre', cross_activation='tanh', data='custom', root_path='./dataset/btc/', data_path='BTC-Hourly.csv', features='S', target='close', freq='h', detail_freq='h', checkpoints='./checkpoints/', seq_len=96, label_len=48, pred_len=96, enc_in=1, dec_in=1, c_out=1, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=[24], factor=3, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=0, itr=1, train_epochs=100, batch_size=1, patience=3, learning_rate=0.0001, des='test', loss='mse', lradj='type1', use_amp=False, use_gpu=True, gpu=0, use_multi_gpu=False, devices='0,1', task_id='custom')


In [15]:
from models.Ensemble import Model as Ensemble
from data_provider.data_factory import data_provider


path = '/home/amirhossein/Projects/FEDformer/checkpoints/custom_Ensemble_random_modes64_custom_ftS_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_test_0'
best_model_path = path + '/' + 'checkpoint.pth'
model = Ensemble(args)

model.load_state_dict(torch.load(best_model_path))


# DataLoader
data_set, data_loader = data_provider(args, 'test')

batch_num = 0
for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(data_loader):
    if i == batch_num:
        batch_x = batch_x.float()
        batch_y = batch_y.float()

        batch_x_mark = batch_x_mark.float()
        batch_y_mark = batch_y_mark.float()

        # decoder input
        dec_inp = torch.zeros_like(batch_y[:, -args.pred_len:, :]).float()
        dec_inp = torch.cat([batch_y[:, :args.label_len, :], dec_inp], dim=1).float()
        pred = model(batch_x, batch_x_mark, dec_inp, batch_y_mark).detach()
        break

Autocorrelation used !
Autocorrelation used !
Autocorrelation used !
Autocorrelation used !
fourier enhanced block used!
modes=64, index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
fourier enhanced block used!
modes=64, index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 26, 27, 28, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70]
 fourier enhanced cross attention used!
modes_q=64, index_q=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 47, 48, 49, 51, 52, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71]
modes_kv=48, index_kv=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 

test 6556


KeyboardInterrupt: 

In [17]:
import matplotlib.pyplot as plt
x = batch_x
y = batch_y

i = 0
f = 0

x_np = data_set.inverse_transform(x[i,:,:].detach().numpy())[:,f]
y_np = data_set.inverse_transform(y[i,-args.pred_len:,:].detach().numpy())[:,f]
pred_np = np.concatenate((x_np[-1:], data_set.inverse_transform(pred[i,:,:].detach().numpy())[:,f]))

# true = np.concatenate([x_np, y_np])
# x_true = np.arange(len(true))
# x_pred = np.arange(len(true)-len(y_np)-1, len(true))
print(x_np.tolist())
print(y_np.tolist())
print(pred_np.tolist())

# plt.figure(figsize=(15, 6))

# plt.plot(x_true, true, label="Real values")
# plt.plot(x_pred, pred_np, label="DLinear")
# plt.legend()
# plt.show()

[3916.540771484375, 3892.909912109375, 3883.741943359375, 3883.999755859375, 3884.870849609375, 3894.310302734375, 3899.700927734375, 3894.730224609375, 3883.890380859375, 3888.460693359375, 3909.441162109375, 3890.909912109375, 3929.999755859375, 3949.611083984375, 3927.900146484375, 3936.999755859375, 3942.560302734375, 3951.739990234375, 3933.140380859375, 3954.060302734375, 3891.210693359375, 3875.290771484375, 3873.919677734375, 3861.550537109375, 3885.909912109375, 3889.950927734375, 3878.271240234375, 3869.941162109375, 3876.029052734375, 3867.628662109375, 3855.989990234375, 3864.198974609375, 3922.001708984375, 3869.800537109375, 3859.810302734375, 3851.980224609375, 3864.999755859375, 3864.849365234375, 3882.448974609375, 3868.310302734375, 3861.339599609375, 3803.579833984375, 3798.968505859375, 3798.759521484375, 3732.169677734375, 3708.300537109375, 3687.888427734375, 3705.269287109375, 3695.370849609375, 3689.659912109375, 3693.771240234375, 3680.989990234375, 3672.980224

[3584.050537109375, 6772.71435546875, 6665.30029296875, 6891.14794921875, 6722.99560546875, 7011.68896484375, 6396.66748046875, 6677.93896484375, 6156.89599609375, 5860.49169921875, 6062.16748046875, 5821.43310546875, 6322.13623046875, 5717.81396484375, 6027.61669921875, 6361.77294921875, 6196.98779296875, 6469.24755859375, 6386.08154296875, 5989.24951171875, 5825.63232421875, 6198.60888671875, 6648.90966796875, 6304.59716796875, 6629.96240234375, 6122.64208984375, 5759.85693359375, 6099.47021484375, 5834.17724609375, 5883.33154296875, 6351.19677734375, 5889.79443359375, 5646.81396484375, 6079.33740234375, 6225.65966796875, 6290.75146484375, 6261.69091796875, 6169.19482421875, 6263.75927734375, 5947.79052734375, 6044.85107421875, 6136.18505859375, 6218.54052734375, 5937.28466796875, 6009.23388671875, 5722.41357421875, 6326.72021484375, 6013.18310546875, 6016.86474609375, 5638.47216796875, 5802.23583984375, 6063.53076171875, 6015.75732421875, 6278.78466796875, 6127.93896484375, 6351.675

In [1]:
from flask import Flask, jsonify
import torch
from torch.utils.data import DataLoader
import time
from models.Ensemble import Model as Ensemble
# from models import FEDformer, Autoformer, Informer, Transformer, Ensemble
import torch.nn as nn
from data_provider.data_factory import data_provider

# from my_dataset import MyDataset

app = Flask(__name__)

path = '/home/amirhossein/Projects/FEDformer/checkpoints/custom_Ensemble_random_modes64_custom_ftS_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_test_0'
best_model_path = path + '/' + 'checkpoint.pth'
model = Ensemble(args)

# if args.use_multi_gpu and args.use_gpu:
#     model = nn.DataParallel(model, device_ids=args.device_ids)
model.load_state_dict(torch.load(best_model_path))

# model.eval()

# DataLoader
data_set, data_loader = data_provider(args, 'test')
# dataset = MyDataset()  
# dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)


global_input_output_pairs = []

@app.route('/get-io-pair', methods=['GET'])
def get_io_pair():
    # Return the most recent input-output pair
    if global_input_output_pairs:
        io_pair = global_input_output_pairs[-1]
        return jsonify({'input': io_pair['input'].tolist(), 'output': io_pair['output'].tolist()})
    else:
        return jsonify({'error': 'No input-output pair available yet.'})

def run_inference_loop():
    # Iterates over the DataLoader
    for inputs in test_loader:
        # Sleep 1 second
        time.sleep(1)
        
        # Generate output from the model
        outputs = model(inputs)
        
        # Store the input-output pair
        global_input_output_pairs.append({
            'input': inputs,
            'output': outputs
        })

if __name__ == '__main__':
    # Start the Flask server in a separate thread
    from threading import Thread
    flask_thread = Thread(target=lambda: app.run(debug=True, use_reloader=False))
    flask_thread.start()
    
    # Run the inference loop
    run_inference_loop()

    # Wait for the Flask server to end
    flask_thread.join()


KeyError: 'c'